# Assistants, 
which encapsulate a base model, instructions, tools, and (context) documents,
# Threads, 
which represent the state of a conversation, and

# Run
Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.



Unlike creating a completion in the Chat Completions API, creating a Run is an asynchronous operation. It will return immediately with the Run's metadata, which includes a status that will initially be set to queued. The status will be updated as the Assistant performs operations (like using tools and adding messages).

To know when the Assistant has completed processing, we can poll the Run in a loop. (Support for streaming is coming soon!) While here we are only checking for a queued or in_progress status, in practice a Run may undergo a variety of status changes which you can choose to surface to the user. (These are called Steps, and will be covered later.)

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [2]:
client : OpenAI = OpenAI()

In [3]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

In [4]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

In [5]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [6]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [7]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

Run(id='run_50ep2WqfNrmV2WK9zU4qHYl9', assistant_id='asst_JxnNh5TJBFjqkJ2BkfeYyJKN', cancelled_at=None, completed_at=1701986068, created_at=1701986063, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1701986063, status='completed', thread_id='thread_KLmMx0Z3Oi42kAP4GfGEfYJ3', tools=[ToolAssistantToolsCode(type='code_interpreter')])


In [8]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[ThreadMessage] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: The solution to the equation 3x + 11 = 14 is x = 1.
